In [1]:
import pandas as pd
import string
!pip install spacy==3.2.3
!pip install -U srsly
!pip install keybert

#!python -m spacy download en
!python -m spacy download en_core_web_sm 
#!python -m spacy download en_core_web_lg
!pip install git+https://github.com/boudinfl/pke.git
!pip install pytextrank

import spacy
nlp=spacy.load('en_core_web_sm')
import nltk
nltk.download('stopwords')
nltk.download('punkt')
from pke.unsupervised import YAKE
from nltk.corpus import stopwords

from spacy.language import Language
import pytextrank

@Language.component("textrank")
def textrank(doc):
    tr = pytextrank.TextRank()
    doc = tr.PipelineComponent(doc)
    return doc
nlp.add_pipe("textrank", last=True)

  Using cached typing_extensions-4.1.1-py3-none-any.whl (26 kB)
  Attempting uninstall: typing-extensions
    Found existing installation: typing-extensions 3.10.0.2
    Uninstalling typing-extensions-3.10.0.2:
      Successfully uninstalled typing-extensions-3.10.0.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
spacy 3.2.3 requires typing-extensions<4.0.0.0,>=3.7.4; python_version < "3.8", but you have typing-extensions 4.1.1 which is incompatible.
aiobotocore 1.3.0 requires botocore<1.20.50,>=1.20.49, but you have botocore 1.24.42 which is incompatible.
     |████████████████████████████████| 13.9 MB 23.4 MB/s            
  Using cached typing_extensions-3.10.0.2-py3-none-any.whl (26 kB)
  Attempting uninstall: typing-extensions
    Found existing installation: typing-extensions 4.1.1
    Uninstalling typing-extensions-4.1.1:
      Successfully uninstalled

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/ec2-user/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /home/ec2-user/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


<function __main__.textrank(doc)>

In [2]:
!pip install PyAthena

In [3]:
%%time
from pyathena import connect
from pyathena.pandas.cursor import PandasCursor

cursor = connect(s3_staging_dir='s3://rd11shiva/',region_name='us-east-1',cursor_class=PandasCursor).cursor()

query='''select *
from "lbathija_db"."normalized_event_pma_after_2018" WHERE
((mdr_text LIKE UPPER('%SEPSIS%'))or(patient LIKE UPPER('%SEPSIS%'))or(product_problems LIKE UPPER('%SEPSIS%'))) and
date_of_event>=(select max(date_of_event) from  "lbathija_db"."normalized_event_pma_after_2018")-600 and date_of_event<(select max(date_of_event) from  "lbathija_db"."normalized_event_pma_after_2018")
'''
event = cursor.execute(query.format()).as_pandas()

CPU times: user 561 ms, sys: 32.1 ms, total: 593 ms
Wall time: 2min 56s


In [4]:
import re
def extract_text(doc:str) -> str:
    if doc=='[]':
        return None
    regex_desc = r"text_type_code=Description of Event or Problem, patient_sequence_number=[0-9], text=([^}]+})"
    desc = list(set(re.findall(regex_desc, doc)))
    desc = ' '.join(desc).replace("}", "").lower()

    regex_addi = r"text_type_code=Additional Manufacturer Narrative, patient_sequence_number=[0-9], text=([^}]+})"
    addi = list(set(re.findall(regex_addi, doc)))
    addi = ' '.join(addi).replace("}", "").lower()

    if desc and addi:
        return desc + " " + addi
    else:
        return desc if desc else addi

In [5]:
event.mdr_text=event.mdr_text.apply(lambda x: extract_text(x))

In [6]:
event_NVN=event[event['product_code']=='NVN']
event_NVN.mdr_text

7      it was reported that this right ventricular (r...
8      it was reported that this right ventricular (r...
13     it was reported that this right atrial (ra) le...
20     boston scientific provided the following infor...
22     it was reported that this right ventricular (r...
                             ...                        
962    it was reported that this right atrial (ra) le...
963    it was reported that this right atrial (ra) le...
964    it was reported that this right atrial (ra) le...
968    it was reported that the patient with this rig...
973    it was reported that this right atrial (ra) le...
Name: mdr_text, Length: 172, dtype: object

In [7]:
event.mdr_text[20]

'boston scientific provided the following information: it was reported that this lead was explanted due to infection with sepsis. an infection was observed following the implantation of this biotronik device. the sterilization process was investigated. the validated process assures that all sterilization parameters, such as gas concentration, temperature, humidity, etc., are within its specified ranges for each distributed device. additionally an analysis of validated microbiological indicators is performed after every sterilization procedure as evidence of successful completion of the sterilization process. review of the biotronik complaint database did not reveal any changes regarding the trend for this type of incident. in summary, the infection was not device related.'

In [8]:
from nltk.tokenize import word_tokenize

def removecustomStopWords(MDR):

    MDRSummaryText = MDR
    tokWordMDR = word_tokenize(MDRSummaryText)

    MDR_stopwords = ['an', 'these', "you're", 'as', 're', 'any', 'is', "aren't", "you'd", 'what', 'down', 'for', 'they', 'yourself', 'before', 'all', 'where', 'won', 'over', 'should', 'each', 'most', 'further', 'having', 'you', 'the', 'd', 'needn', 'do', 'had', 'shouldn', 'no', 'such', 'there', 'who', 'a', 'my', 'will', 'doing', 'now', 'y', 'between', 'if', 'other', 'so', 's', 'doesn', 'he', 'was', 'this', "mightn't", 'until', 'again', 'has', 'ain', 'didn', 'under', 'mightn', 'that', 'our', 'them', 'itself', 'because', 'some', 'yourselves', 't', 'wouldn', 'him', 'your', 'on', 'in', 'just', "she's", "haven't", 'been', 'll', 'her', "don't", 'we', 'off', 'same', "doesn't", 'or', 'aren', 'once', 'mustn', 'am', 'it', 'of', 'ourselves', 'couldn', "wasn't", 'whom', "should've", 'which', 'ma', 'did', 'why', "weren't", "that'll", 'but', 'and', 'after', "didn't", 'with', "hasn't", 'being', 'out', 'more', "shouldn't", "mustn't", 'through', 'when', 'does', "needn't", "shan't", 'wasn', 'during', 'are', 'at', 'few', "won't", 'don', 'ours', 'to', 'herself', 'isn', 'himself', 'hers', 'were', 'above', 'both', 'against', "wouldn't", 'shan', "it's", 'those', 'own', 'up', 'below', 'yours', 'its', 'about', 'very', "isn't", 'from', 'she', 'how', 'o', 'his', 'here', 'while', 've', 'hasn', 'can', 'me', "you'll", 'then', 'nor', 'm', 'themselves', 'haven', 'theirs', 'into', 'i', 'only', "couldn't", 'than', 'by', 'their', 'be', "hadn't", 'weren', 'too', "you've", 'have', 'hadn', 'myself']
    
    MDRSummary_noSW = list()

    for i in range(0,len(tokWordMDR)):

        if tokWordMDR[i].lower() in MDR_stopwords: 
            pass
        else:
            MDRSummary_noSW.append(tokWordMDR[i])

    return (" ").join(MDRSummary_noSW)

In [9]:
event_NVN['mdr_text']=event_NVN.mdr_text.apply(lambda x: removecustomStopWords(x))


/home/ec2-user/anaconda3/envs/pytorch_latest_p36/lib/python3.6/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':


In [10]:
event_NVN.mdr_text[20]

'boston scientific provided following information : reported lead explanted due infection sepsis . infection observed following implantation biotronik device . sterilization process investigated . validated process assures sterilization parameters , gas concentration , temperature , humidity , etc. , within specified ranges distributed device . additionally analysis validated microbiological indicators performed every sterilization procedure evidence successful completion sterilization process . review biotronik complaint database not reveal changes regarding trend type incident . summary , infection not device related .'

In [11]:
pd.options.mode.chained_assignment = None # default='warn'

In [12]:
%%time
event_NVN.reset_index(inplace=True)

event_NVN['pytextrank_keyphrases'] = ''
event_NVN['pytextrank_keywords'] = ''

for i in range(len(event_NVN)):
    doc =  event_NVN['mdr_text'][i]
    doc1 = nlp(doc)
    m = []
    c = 20
    for p in doc1._.phrases:

        m.append(p.text)
        c = c - 1
        if c < 1:
            break
    words = [word for word in m if len(word.split()) == 1]

    event_NVN['pytextrank_keyphrases'][i] = m
    event_NVN['pytextrank_keywords'][i]= words

CPU times: user 2.8 s, sys: 3.89 ms, total: 2.81 s
Wall time: 2.81 s


In [13]:
event_NVN.pytextrank_keywords.explode().value_counts().head(20)

information    158
future         139
rv              46
device          24
infection       20
changes         19
temperature     19
boston          18
humidity        14
analysis        11
pacemaker        8
patient          6
revision         5
2017865          5
lead             4
hematoma         3
product          3
6                3
order            2
use              2
Name: pytextrank_keywords, dtype: int64

In [14]:
event_NVN.pytextrank_keyphrases.explode().value_counts().head(20)

information                            158
future                                 139
supplemental report                    139
additional adverse patient effects     136
part system revision                   118
due infection sepsis                   116
( ra                                    85
ra lead                                 82
intravenous antibiotics                 66
right ventricular                       48
rv                                      46
rv lead                                 46
device                                  24
infection                               20
gas concentration                       19
specified ranges                        19
boston scientific                       19
trend type incident                     19
review biotronik complaint database     19
changes                                 19
Name: pytextrank_keyphrases, dtype: int64

In [15]:
event_NVN.product_problems.explode().value_counts()

[Adverse Event Without Identified Device or Use Problem]    167
[Material Integrity Problem]                                  1
[Failure to Capture, Pacing Problem]                          1
[Difficult to Remove]                                         1
[Failure to Capture]                                          1
[Migration]                                                   1
Name: product_problems, dtype: int64

In [16]:
import re
def extract_patient(doc:str) -> str:
    if doc=='[]':
        return None
    regex_desc = r"patient_problems=([^}]+})"
    desc = list(set(re.findall(regex_desc, doc)))
    desc = ' '.join(desc).replace("}", "").lower()
    return desc if desc else addi

In [17]:
event_NVN['patient']=event_NVN.patient.apply(lambda x: extract_patient(x))

In [18]:
event_NVN.apply(lambda row:  row['patient'].strip('[]').replace('"', '').replace(' ', '').split(',')   , axis=1).explode().value_counts()


unspecifiedinfection    167
sepsis                  162
bacterialinfection        5
pocketerosion             5
hematoma                  2
arrhythmia                2
fever                     2
fluiddischarge            2
purulentdischarge         2
rash                      1
embolism/embolus          1
wounddehiscence           1
cardiacarrest             1
pain                      1
swelling/edema            1
dtype: int64